## Importação das bibliotecas

In [ ]:
# Importando a biblioteca pandas, importante para tratamento dos dados importados
import pandas as pd

# Importando a biblioteca pyplot e seaborn, importante para visualização dos dados importados
import matplotlib.pyplot as plt
import seaborn as sns

# Importando a biblioteca requests, para requisitar a fonte dos dados (Github)
import requests

# Importando bibliotecas io e ftfy, para tratamento de codificação de caracteres truncados (utf-8)
from io import StringIO
import ftfy

## Carregamento dos dados

In [ ]:
# Carregando dados diretamento do github e corrigindo problemas de codificação
url = requests.get('https://raw.githubusercontent.com/Ada-Empregabilidade/adahack-2024-dados/main/base_dados/base_de_dados.csv').content.decode('utf-8')
linhas = url.split('\n')
lista_linhas = [ftfy.fix_text(linha) for linha in linhas]
data = StringIO('\n'.join(lista_linhas))
df = pd.read_csv(data)

_Optamos por utilizar as informaçoes diretamente da fonte de dados, ao invés de baixa-las e adiciona-las manualmente. O arquivo csv fornecido contém problemas de codificação e também optamos por fazer o tratamento dentro do notebook._

## Exploração inicial dos dados

##### Primeiro contato com os dados

In [ ]:
# Comando head para exibir as 10 primeiras linhas do dataset
df.head(10)

_Aqui já encontramos algumas anomalias, que serão tratadas ao longo do processo de limpeza, tais como:_
1. _Coluna 'id' e 'id.1' duplicadas._
2. _Coluna 'Idade' com valores negativos._
3. _'Endereço' estadunidense com uma quebra de linha não reconhecida_
4. _'Formação' possui valores nulos._
5. _Coluna 'Tempo de casa' com valores negativos_
6. _Coluna 'Senioridade' com diferentes grafias para um mesmo cargo_

##### Compreendendo as variáveis qualitativas

In [ ]:
#Valores únicos para a variável 'Raça'
df['Raça'].unique()

In [ ]:
#Valores únicos para a variável 'Genero'
df['Genero'].unique()

In [ ]:
#Valores únicos para a variável 'Formação'
df['Formação'].unique()

In [ ]:
#Valores únicos para a variável 'Estado'
df['Estado'].unique()

In [ ]:
#Valores únicos para a variável 'Departamento'
df['Departamento'].unique()

In [ ]:
#Valores únicos para a variável 'Senioridade'
df['Senioridade'].unique()

_Podemos compreender os diferentes tipos de variáveis, e como estão distribuidas ao longo do dataset_

##### Tamanho da base de dados

In [ ]:
#Entendendo o tamanho da base de dados
print(f'O presente dataset posssui {df.shape[0]} linhas e {df.shape[1]} colunas. ')

In [ ]:
# Listando colunas
df.columns

##### Estatística Descritiva

In [ ]:
# Panorama estatístico das colunas numéricas do dataset
df.describe()

In [ ]:
# Panorama estatístico das colunas não-numéricas do dataset
df_nao_numerico = df.select_dtypes(exclude='number')
df_nao_numerico.describe()

##### Tipos de variáveis

In [ ]:
# Verificando tipos de variáveis por cada coluna.
tipos_de_dados = df.info()
display(tipos_de_dados)

_Acima percebemos que os tipos de dados estão praticamente conformes, exceto pela coluna 'Idade', que não precisa ser do tipo float e deve ser um número inteiro. O mesmo acontece com a coluna 'Tempo de casa'._

## Limpeza e preparação dos dados

##### Removendo linhas duplicadas

In [ ]:
#remover duplicatas
df = df.drop_duplicates()

In [ ]:
#Remover colunas desnecessárias
df = df.drop(columns=['id', 'id.1', 'Nome', 'Endereço'])

##### Lidando com dados faltantes

In [ ]:
#Relacionando dados faltantes
df.isna().sum()

_O número de linhas com valores nulos é no total 300, o que representa 3% da quantidade de linhas totais do dataset, por isso iremos retirá-las ao invés de preencher com valores tais como média, mediana, etc._

Sugestão Iago:
Considerando que esses dados faltantes, como de atributos diferentes temos: 56+44+200 = 300. Como temos um total de 10000 observações é razoalvel eliminar esses dados. Representa eliminasr 300/10000, ou seja, 3% dos dados.

In [ ]:
#Deletar dados faltantes
df = df.dropna(axis=0, how='any', inplace=False)

In [ ]:
#Conferindo resultados
df.isna().sum()

Outliers

In [ ]:
#Boxplot

# Assuming df is your DataFrame containing numerical columns
numerical_columns = df.select_dtypes(include='number').columns

plt.figure(figsize=(10, 6))  # Adjust the figure size if needed
for column in numerical_columns:
    df.boxplot(column=column)
    plt.title(f'Boxplot for {column}')
    plt.show()

Não faz sentido funcionários com idade menor que 18 anos. Não faz sentido tempo de casa negativo.

Avaliar a possibilidade de eliminar esses casos.

##### Removendo dados inválidos

In [ ]:
# Encontrando tempos de experiência negativos
tempo_casa_negativo = df.loc[df['Tempo de casa'] < 0]
display(tempo_casa_negativo)

In [ ]:
df.shape

Sugestão Iago:

Analisando a coluna 'Tempo de casa' encontramos 96 observações com valor negativo. O que não parece fazer sentido. Como repesentam 96/9701 = 0.01, ou seja, 1% é razoável eliminar essas linhas.

In [ ]:
# Encontrando tempos de casa zerados
tempo_casa_zero = df.loc[df['Tempo de casa'] == 0]
display(tempo_casa_zero)

_Os valores zerados para a variável 'Tempo de casa', não serão excluídas, porque entendemos não serem inválidas, apenas que os funcionários não alcançaram 1 (um) ano._

In [ ]:
df = df.drop(df[df['Tempo de casa'] < 0 ].index)
df.shape

In [ ]:
#Conferindo resultados
tempo_casa_negativo = df.loc[df['Tempo de casa'] < 0]
display(tempo_casa_negativo)

In [ ]:
# Encontrando Idades inválidas (menores de idade)
menor_idade = df.loc[df['Idade'] < 18]
display(menor_idade)

In [ ]:
df.shape

Sugestão Iago:

Analisando a coluna Idade junto a coluna Formação, não faz sentido esses dados com Idade menor que 18 anos. Como repesentam 12/9605 = 0.001, ou seja, 0,1% é razoázvel eliminar essas linhas.

In [ ]:

df = df.drop(df[df['Idade'] < 18].index)

In [ ]:
#Conferindo resultados
menor_idade = df.loc[df['Idade'] < 18]
display(menor_idade)

In [ ]:
tempo_casa_idade = df[df['Tempo de casa'] > df['Idade']]
display(tempo_casa_idade)

In [ ]:
tempo_casa_alto = df[df['Tempo de casa'] >= 17]
display(tempo_casa_alto)

In [ ]:
tempo_casa_menor_idade = df[(df['Idade'] - df['Tempo de casa']) < 18]
display(tempo_casa_menor_idade)

# *TEMOS UM PROBLEMA AQUI*

In [ ]:
850/2

Sugestão Iago:

Explicar.

## Manipulação e transformação dos dados

##### Tipos de variáveis

In [ ]:
# Verificando tipos de variáveis das colunas
df.info()

In [ ]:
#Verificando se realmente existem números decimais nas colunas 'Idade' e 'Tempo de casa'
df_decimais = df.loc[(df['Idade'] % 1 != 0) & (df['Tempo de casa'] % 1 != 0)]
df_decimais.info()

In [ ]:
# Convertendo as colunas Idade e Tempo de Casa para (INT)
df['Idade'] = df['Idade'].astype(int)
df['Tempo de casa'] = df['Tempo de casa'].astype(int)

In [ ]:
# Conferindo resultados
df.info()

##### Corrigindo grafias

In [ ]:
df['Senioridade'] = df['Senioridade'].replace('gerente', 'Gerente')

In [ ]:
df['Genero'] = df['Genero'].replace({'Fem' : 'Feminino', 'Masc' : 'Masculino'})

In [ ]:
df['Raça'] = df['Raça'].replace({'preto' : 'Preto', 'pardo' : 'Pardo', 'branco' : 'Branco', 'indígena' : 'Indígena', 'amarelo' : 'Amarelo'})

In [ ]:
df = df.rename(columns={'Genero': 'Gênero', 'Tempo de casa' : 'Tempo de Empresa'})

##### Coluna diversidade

In [ ]:
# Criando coluna Diversidade para facilitar a visualização das diversidades presentes
df['Diversidade'] = df ['Raça'] != 'branco'

##### Organização

In [ ]:
# Alterando a ordem das colunas para uma melhor visualização
nova_ordem_colunas = ['Gênero', 'Idade', 'Raça', 'Diversidade', 'Formação', 'Estado', 'Tempo de Empresa', 'Departamento', 'Senioridade']
df = df.reindex(columns=nova_ordem_colunas)

In [ ]:
#Conferindo resultados das transformações
df.head(10)

## Análise e exploração dos dados

#### Como os colaboradores estão distribuídos de acordo com o gênero?

In [ ]:
contagem_genero = df['Gênero'].value_counts()
sns.set(style="whitegrid")

plt.figure(figsize=(8, 6))
barras = sns.barplot(x=contagem_genero.index, y=contagem_genero.values, palette="pastel", hue=contagem_genero.index, dodge=False, legend=False)

plt.title('Distribuição dos Colaboradores x Gênero')
plt.xlabel('Gênero')
plt.ylabel('Colaboradores')

for barra in barras.patches:
    barras.annotate(format(barra.get_height(), '.0f'), 
                   (barra.get_x() + barra.get_width() / 2, 
                    barra.get_height()), 
                   ha='center', va='center',
                   size=12, xytext=(0, 5),
                   textcoords='offset points')

total = df.shape[0]
for barra in barras.patches:
    barras.annotate(f'{barra.get_height() / total:.2%}', 
                   (barra.get_x() + barra.get_width() / 2, 
                    barra.get_height() / 2), 
                   ha='center', va='center',
                   size=20, xytext=(0, 0),
                   textcoords='offset points', color='white')
plt.show()

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(8, 6))
histograma = sns.histplot(data=df, x='Idade', bins=10, kde=True)

plt.title('Distribuição de Idade')
plt.xlabel('Idade')
plt.ylabel('Número de Colaboradores')

plt.show()

#### Identificar como é a distribuição de idade dos colaboradores.

In [ ]:
contagem_raca = df['Raça'].value_counts()
sns.set(style="whitegrid")

plt.figure(figsize=(8, 6))
barras = sns.barplot(x=contagem_raca.index, y=contagem_raca.values, hue=contagem_raca.index, palette="pastel")

plt.title('Distribuição dos Colaboradores por Raça')
plt.xlabel('Raça')
plt.ylabel('Número de Colaboradores')

for barra in barras.patches:
    barras.annotate(format(barra.get_height(), '.0f'), 
                   (barra.get_x() + barra.get_width() / 2, 
                    barra.get_height()), 
                   ha='center', va='center',
                   size=12, xytext=(0, 5),
                   textcoords='offset points')

plt.show()

In [ ]:
contagem_escolaridade = df['Formação'].value_counts()
sns.set(style="whitegrid")

plt.figure(figsize=(8, 6))
barras = sns.barplot(x=contagem_escolaridade.index, y=contagem_escolaridade.values, hue=contagem_escolaridade.index, palette="pastel")

plt.title('Distribuição dos Colaboradores por Nível de Escolaridade')
plt.xlabel('Nível de Escolaridade')
plt.ylabel('Número de Colaboradores')

for barra in barras.patches:
    barras.annotate(format(barra.get_height(), '.0f'), 
                   (barra.get_x() + barra.get_width() / 2, 
                    barra.get_height()), 
                   ha='center', va='center',
                   size=12, xytext=(0, 5),
                   textcoords='offset points')

plt.show()

#### Como os colaboradores estão distribuídos de acordo com região(Nordeste, Sul, etc)?

In [ ]:
regioes = {
    'Acre': 'Norte', 'Alagoas': 'Nordeste', 'Amapá': 'Norte', 'Amazonas': 'Norte', 'Bahia': 'Nordeste', 'Ceará': 'Nordeste',
    'Distrito Federal': 'Centro-Oeste', 'Espírito Santo': 'Sudeste', 'Goiás': 'Centro-Oeste', 'Maranhão': 'Nordeste', 'Mato Grosso': 'Centro-Oeste',
    'Mato Grosso do Sul': 'Centro-Oeste', 'Minas Gerais': 'Sudeste', 'Pará': 'Norte', 'Paraíba': 'Nordeste', 'Paraná': 'Sul', 'Pernambuco': 'Nordeste',
    'Piauí': 'Nordeste', 'Rio de Janeiro': 'Sudeste', 'Rio Grande do Norte': 'Nordeste', 'Rio Grande do Sul': 'Sul', 'Rondônia': 'Norte', 'Roraima': 'Norte', 
    'Santa Catarina': 'Sul', 'São Paulo': 'Sudeste', 'Sergipe': 'Nordeste', 'Tocantins': 'Norte'
}

df['Região'] = df['Estado'].map(regioes)

contagem_regiao = df['Região'].value_counts()


sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.barplot(x=contagem_regiao.index, y=contagem_regiao.values, hue=contagem_regiao.index, palette="pastel")
plt.title('Distribuição dos Colaboradores por Região do Brasil')
plt.xlabel('Região')
plt.ylabel('Número de Colaboradores')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

#### Como os colaboradores estão distribuídos de acordo com o Estado?

In [ ]:
uf = {
        'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM', 'Bahia': 'BA',
        'Ceará': 'CE', 'Distrito Federal': 'DF', 'Espírito Santo': 'ES', 'Goiás': 'GO',
        'Maranhão': 'MA', 'Mato Grosso': 'MT', 'Mato Grosso do Sul': 'MS', 'Minas Gerais': 'MG',
        'Pará': 'PA', 'Paraíba': 'PB', 'Paraná': 'PR', 'Pernambuco': 'PE', 'Piauí': 'PI',
        'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN', 'Rio Grande do Sul': 'RS',
        'Rondônia': 'RO', 'Roraima': 'RR', 'Santa Catarina': 'SC', 'São Paulo': 'SP',
        'Sergipe': 'SE', 'Tocantins': 'TO'
}
    
df['UF'] = df['Estado'].map(uf)
contagem_estado = df['UF'].value_counts()

sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.barplot(x=contagem_estado.index, y=contagem_estado.values, hue=contagem_estado.index, palette="pastel")
plt.title('Distribuição dos Colaboradores por Estado do Brasil')
plt.xlabel('Estado')
plt.ylabel('Número de Colaboradores')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### Visualização dos resultados

### Comunicação dos resultados

In [ ]:
df.to_csv('tratado.csv')

In [ ]:
df